In [7]:
import pandas as pd

# Clean the annual returns file
def clean_annual_file(input_path, output_path):
    # Read the file, using second row as header
    df = pd.read_excel(input_path, header=1)

    # Get all columns
    all_columns = df.columns.tolist()

    # Find the year columns (2024-2014) - they should be after "Scheme Name" and some other columns
    year_columns = [col for col in all_columns if str(col).isdigit() and 2014 <= int(col) <= 2024]

    # Select only Scheme Name and the year columns
    required_cols = ['Scheme Name'] + sorted(year_columns, reverse=True)
    df = df[required_cols]

    # Replace '-' with None
    df = df.replace('-', None)

    # Save cleaned file
    df.to_excel(output_path, index=False)
    print(f"Cleaned annual file saved with columns: {required_cols}")

# Clean the monthly returns file
def clean_monthly_file(input_path, output_path):
    df = pd.read_excel(input_path, header=1)

    # Manually specify the exact monthly columns needed
    monthly_cols = ["May'25", "Apr'25", "Mar'25", "Feb'25",
                   "Jan'25", "Dec'24", "Nov'24", "Oct'24",
                   "Sep'24", "Aug'24", "Jul'24"]

    # Verify these columns exist
    available_cols = [col for col in monthly_cols if col in df.columns]

    # Select only Scheme Name and available monthly columns
    df = df[['Scheme Name'] + available_cols]
    df.to_excel(output_path, index=False)

# File paths
annual_input = '/content/Annual Returns - Mutual fund screener Mutual Fund Screening and Analysis Tool - Moneycontrol.com.xlsx'
monthly_input = '/content/Monthly Returns - Mutual fund screener Mutual Fund Screening and Analysis Tool - Moneycontrol.com.xlsx'

annual_output = 'cleaned_annual_returns.xlsx'
monthly_output = 'cleaned_monthly_returns.xlsx'

# Clean both files
clean_annual_file(annual_input, annual_output)
clean_monthly_file(monthly_input, monthly_output)

Cleaned annual file saved with columns: ['Scheme Name', 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014]


In [10]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from sklearn.preprocessing import MinMaxScaler

# # Load data (WITHOUT setting index during read)
# annual_df = pd.read_excel('cleaned_annual_returns.xlsx')
# monthly_df = pd.read_excel('cleaned_monthly_returns.xlsx')

# # Explicitly set index after loading
# annual_df = annual_df.set_index('Scheme Name')
# monthly_df = monthly_df.set_index('Scheme Name')

# # Verify data loading
# print("Annual Data Sample:")
# print(annual_df.head())
# print("\nMonthly Data Sample:")
# print(monthly_df.head())

# # Prepare sequences for each fund
# results = []
# for fund_name in annual_df.index:
#     try:
#         # Get annual and monthly data for this fund
#         annual_data = annual_df.loc[fund_name].values  # Gets all annual returns
#         monthly_data = monthly_df.loc[fund_name, :"Jul'24"].values  # Gets all monthly returns except May'25

#         # Combine features and get target
#         X = np.concatenate([annual_data, monthly_data])
#         y = monthly_df.loc[fund_name, "May'25"]

#         # Skip if any missing data
#         if np.isnan(X).any() or np.isnan(y):
#             continue

#         # Normalize
#         scaler = MinMaxScaler()
#         X_scaled = scaler.fit_transform(X.reshape(-1, 1)).flatten()
#         y_scaled = scaler.transform([[y]]).flatten()[0]

#         # Reshape for TFT (1 sample, timesteps, 1 feature)
#         X_reshaped = X_scaled.reshape(1, len(X), 1)

#         # Build simple TFT model
#         inputs = layers.Input(shape=(len(X), 1))
#         x = layers.LSTM(32, return_sequences=True)(inputs)
#         x = layers.LSTM(16)(x)
#         x = layers.Dense(8, activation='relu')(x)
#         outputs = layers.Dense(1)(x)
#         model = models.Model(inputs, outputs)
#         model.compile(optimizer='adam', loss='mse')

#         # Train (just on this fund's data)
#         model.fit(X_reshaped, np.array([y_scaled]), epochs=50, verbose=0)

#         # Predict
#         pred_scaled = model.predict(X_reshaped, verbose=0)[0][0]
#         pred = scaler.inverse_transform([[pred_scaled]])[0][0]

#         results.append({
#             'Fund': fund_name,
#             'Actual': y,
#             'Predicted': pred,
#             'Error': abs(pred - y)
#         })
#     except Exception as e:
#         print(f"Skipping {fund_name} due to error: {str(e)}")
#         continue

# # Create and display results
# if results:
#     results_df = pd.DataFrame(results)
#     print("\nPrediction Results:")
#     print(results_df[['Fund', 'Actual', 'Predicted', 'Error']].sort_values('Error'))

#     mae = results_df['Error'].mean()
#     print(f"\nMean Absolute Error: {mae:.4f}")
# else:
#     print("No valid funds processed - check data loading")

Annual Data Sample:
                                                      2024    2023    2022  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.1960  0.3809  0.1363   
Invesco India Contra Fund - Direct Plan - Growth    0.3597  0.2958  0.0509   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2256  0.2672  0.0773   
Franklin India ELSS Tax Saver Fund - Direct Pla...  0.2321  0.3236  0.0627   
HDFC ELSS Tax saver - Direct Plan - Growth          0.2196  0.3348  0.1114   

                                                      2021    2020    2019  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.4911  0.3061 -0.0034   
Invesco India Contra Fund - Direct Plan - Growth    0.2949  0.2220  0.0710   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2354  0.1343 -0.0072   
Franklin India ELSS Tax Saver Fund - Direct


Prediction Results:
                                                 Fund  Actual  Predicted  \
39  Invesco India Mid Cap Fund - Direct Plan - Growth  0.0753   0.075325   
8        LIC MF ELSS Tax Saver - Direct Plan - Growth  0.0294   0.029436   
56  ICICI Prudential Value Fund - Direct Plan - Gr...  0.0250   0.025082   
14  Parag Parikh Flexi Cap Fund - Direct Plan - Gr...  0.0271   0.026850   
4          HDFC ELSS Tax saver - Direct Plan - Growth  0.0158   0.015157   
0              SBI Contra Fund - Direct Plan - Growth  0.0236   0.024388   
28         Kotak Bluechip Fund - Direct Plan - Growth  0.0259   0.025111   
37           HDFC Mid Cap Fund - Direct Plan - Growth  0.0586   0.059409   
34    Edelweiss Large Cap Fund - Direct Plan - Growth  0.0220   0.022878   
10         HDFC Flexi Cap Fund - Direct Plan - Growth  0.0137   0.012615   
1    Invesco India Contra Fund - Direct Plan - Growth  0.0301   0.028955   
35  Canara Robeco Large Cap Fund - Direct Plan - G...  0.0209   0.0

In [11]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from sklearn.preprocessing import MinMaxScaler

# # Load data (WITHOUT setting index during read)
# annual_df = pd.read_excel('cleaned_annual_returns.xlsx')
# monthly_df = pd.read_excel('cleaned_monthly_returns.xlsx')

# # Explicitly set index after loading
# annual_df = annual_df.set_index('Scheme Name')
# monthly_df = monthly_df.set_index('Scheme Name')

# # Verify data loading
# print("Annual Data Sample:")
# print(annual_df.head())
# print("\nMonthly Data Sample:")
# print(monthly_df.head())

# # Prepare sequences for each fund
# results = []
# for fund_name in annual_df.index:
#     try:
#         # Get all available data
#         annual_data = annual_df.loc[fund_name].values
#         monthly_data = monthly_df.loc[fund_name].values  # Includes May'25

#         X = np.concatenate([annual_data, monthly_data])

#         if np.isnan(X).any():
#             continue

#         # Normalize and reshape
#         scaler = MinMaxScaler()
#         X_scaled = scaler.fit_transform(X.reshape(-1, 1)).flatten()
#         X_reshaped = X_scaled.reshape(1, len(X), 1)

#         # Build and train model (same architecture)
#         inputs = layers.Input(shape=(len(X), 1))
#         x = layers.LSTM(32, return_sequences=True)(inputs)
#         x = layers.LSTM(16)(x)
#         x = layers.Dense(8, activation='relu')(x)
#         outputs = layers.Dense(1)(x)
#         model = models.Model(inputs, outputs)
#         model.compile(optimizer='adam', loss='mse')
#         model.fit(X_reshaped, np.array([0]), epochs=50, verbose=0)  # Dummy target

#         # Predict June'25
#         pred_scaled = model.predict(X_reshaped, verbose=0)[0][0]
#         pred = scaler.inverse_transform([[pred_scaled]])[0][0]

#         results.append({
#             'Fund': fund_name,
#             'Predicted_Jun_25': pred  # Only prediction available
#         })
#     except Exception as e:
#         print(f"Skipping {fund_name}: {str(e)}")
#         continue

# # Display predictions
# if results:
#     results_df = pd.DataFrame(results)
#     print("\nJune 2025 Return Predictions:")
#     print(results_df.sort_values('Predicted_Jun_25', ascending=False))
# else:
#     print("No valid predictions generated")

Annual Data Sample:
                                                      2024    2023    2022  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.1960  0.3809  0.1363   
Invesco India Contra Fund - Direct Plan - Growth    0.3597  0.2958  0.0509   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2256  0.2672  0.0773   
Franklin India ELSS Tax Saver Fund - Direct Pla...  0.2321  0.3236  0.0627   
HDFC ELSS Tax saver - Direct Plan - Growth          0.2196  0.3348  0.1114   

                                                      2021    2020    2019  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.4911  0.3061 -0.0034   
Invesco India Contra Fund - Direct Plan - Growth    0.2949  0.2220  0.0710   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2354  0.1343 -0.0072   
Franklin India ELSS Tax Saver Fund - Direct

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler

# Load data (WITHOUT setting index during read)
annual_df = pd.read_excel('cleaned_annual_returns.xlsx')
monthly_df = pd.read_excel('cleaned_monthly_returns.xlsx')

# Explicitly set index after loading
annual_df = annual_df.set_index('Scheme Name')
monthly_df = monthly_df.set_index('Scheme Name')

# Get all monthly columns except May'25 (which is our target)
monthly_feature_columns = [col for col in monthly_df.columns if col != "May'25"]

# Verify data loading
print("Annual Data Sample:")
print(annual_df.head())
print("\nMonthly Features Sample (excluding May'25):")
print(monthly_df[monthly_feature_columns].head())

# Prepare sequences for each fund
results = []
for fund_name in annual_df.index:
    try:
        # Get annual data and monthly data EXCLUDING May'25
        annual_data = annual_df.loc[fund_name].values
        monthly_data = monthly_df.loc[fund_name, monthly_feature_columns].values

        # Combine features and get target (May'25)
        X = np.concatenate([annual_data, monthly_data])
        y = monthly_df.loc[fund_name, "May'25"]

        # Skip if any missing data
        if np.isnan(X).any() or np.isnan(y):
            continue

        # Normalize
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X.reshape(-1, 1)).flatten()
        y_scaled = scaler.transform([[y]]).flatten()[0]

        # Reshape for TFT (1 sample, timesteps, 1 feature)
        X_reshaped = X_scaled.reshape(1, len(X), 1)

        # Build simple TFT model
        inputs = layers.Input(shape=(len(X), 1))
        x = layers.LSTM(32, return_sequences=True)(inputs)
        x = layers.LSTM(16)(x)
        x = layers.Dense(8, activation='relu')(x)
        outputs = layers.Dense(1)(x)
        model = models.Model(inputs, outputs)
        model.compile(optimizer='adam', loss='mse')

        # Train (just on this fund's data)
        model.fit(X_reshaped, np.array([y_scaled]), epochs=50, verbose=0)

        # Predict
        pred_scaled = model.predict(X_reshaped, verbose=0)[0][0]
        pred = scaler.inverse_transform([[pred_scaled]])[0][0]

        results.append({
            'Fund': fund_name,
            'Actual': y,
            'Predicted': pred,
            'Error': abs(pred - y)
        })
    except Exception as e:
        print(f"Skipping {fund_name} due to error: {str(e)}")
        continue

# Create and display results
if results:
    results_df = pd.DataFrame(results)
    print("\nPrediction Results (May'25 Actual vs Predicted):")
    print(results_df[['Fund', 'Actual', 'Predicted', 'Error']].sort_values('Error'))

    mae = results_df['Error'].mean()
    print(f"\nMean Absolute Error: {mae:.4f}")
    print(f"Median Absolute Error: {results_df['Error'].median():.4f}")
    print(f"Accuracy within ±0.02: {(results_df['Error'] <= 0.02).mean()*100:.1f}%")
else:
    print("No valid funds processed - check data loading")

Annual Data Sample:
                                                      2024    2023    2022  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.1960  0.3809  0.1363   
Invesco India Contra Fund - Direct Plan - Growth    0.3597  0.2958  0.0509   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2256  0.2672  0.0773   
Franklin India ELSS Tax Saver Fund - Direct Pla...  0.2321  0.3236  0.0627   
HDFC ELSS Tax saver - Direct Plan - Growth          0.2196  0.3348  0.1114   

                                                      2021    2020    2019  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.4911  0.3061 -0.0034   
Invesco India Contra Fund - Direct Plan - Growth    0.2949  0.2220  0.0710   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2354  0.1343 -0.0072   
Franklin India ELSS Tax Saver Fund - Direct

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler

# Load data (WITHOUT setting index during read)
annual_df = pd.read_excel('cleaned_annual_returns.xlsx')
monthly_df = pd.read_excel('cleaned_monthly_returns.xlsx')

# Explicitly set index after loading
annual_df = annual_df.set_index('Scheme Name')
monthly_df = monthly_df.set_index('Scheme Name')

# Get all monthly columns except Jun'25 (which is our new target)
monthly_feature_columns = [col for col in monthly_df.columns if col != "Jun'25"]

print("Annual Data Sample:")
print(annual_df.head())
print("\nMonthly Features Sample (excluding Jun'25):")
print(monthly_df[monthly_feature_columns].head())

# Prepare sequences for each fund
results = []
for fund_name in annual_df.index:
    try:
        # Get annual and monthly data EXCLUDING Jun'25
        annual_data = annual_df.loc[fund_name].values
        monthly_data = monthly_df.loc[fund_name, monthly_feature_columns].values

        # Combine features
        X = np.concatenate([annual_data, monthly_data])

        # We try to get target (Jun'25), but it's okay if it's missing
        y = monthly_df.loc[fund_name, "Jun'25"] if "Jun'25" in monthly_df.columns else np.nan

        # Skip if input data is missing
        if np.isnan(X).any():
            continue

        # Normalize
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X.reshape(-1, 1)).flatten()

        # If actual value exists, scale it too
        y_scaled = scaler.transform([[y]]).flatten()[0] if not np.isnan(y) else None

        # Reshape for TFT
        X_reshaped = X_scaled.reshape(1, len(X), 1)

        # Build model
        inputs = layers.Input(shape=(len(X), 1))
        x = layers.LSTM(32, return_sequences=True)(inputs)
        x = layers.LSTM(16)(x)
        x = layers.Dense(8, activation='relu')(x)
        outputs = layers.Dense(1)(x)
        model = models.Model(inputs, outputs)
        model.compile(optimizer='adam', loss='mse')

        # If actual value available, train on it
        if y_scaled is not None:
            model.fit(X_reshaped, np.array([y_scaled]), epochs=50, verbose=0)
        else:
            model.fit(X_reshaped, X_reshaped[:, -1, :], epochs=50, verbose=0)  # dummy train

        # Predict
        pred_scaled = model.predict(X_reshaped, verbose=0)[0][0]
        pred = scaler.inverse_transform([[pred_scaled]])[0][0]

        results.append({
            'Fund': fund_name,
            'Actual': y if not np.isnan(y) else 'N/A',
            'Predicted': pred,
            'Error': abs(pred - y) if not np.isnan(y) else 'N/A'
        })
    except Exception as e:
        print(f"Skipping {fund_name} due to error: {str(e)}")
        continue

# Display predictions
if results:
    results_df = pd.DataFrame(results)
    print("\nPrediction Results (Jun'25):")
    print(results_df[['Fund', 'Actual', 'Predicted', 'Error']])

    if results_df['Error'].dtype != 'O':  # Only if error is numeric
        mae = results_df['Error'].mean()
        print(f"\nMean Absolute Error: {mae:.4f}")
        print(f"Median Absolute Error: {results_df['Error'].median():.4f}")
        print(f"Accuracy within ±0.02: {(results_df['Error'] <= 0.02).mean()*100:.1f}%")
else:
    print("No valid funds processed.")


Annual Data Sample:
                                                      2024    2023    2022  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.1960  0.3809  0.1363   
Invesco India Contra Fund - Direct Plan - Growth    0.3597  0.2958  0.0509   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2256  0.2672  0.0773   
Franklin India ELSS Tax Saver Fund - Direct Pla...  0.2321  0.3236  0.0627   
HDFC ELSS Tax saver - Direct Plan - Growth          0.2196  0.3348  0.1114   

                                                      2021    2020    2019  \
Scheme Name                                                                  
SBI Contra Fund - Direct Plan - Growth              0.4911  0.3061 -0.0034   
Invesco India Contra Fund - Direct Plan - Growth    0.2949  0.2220  0.0710   
Quantum ELSS Tax Saver Fund - Direct Plan - Growth  0.2354  0.1343 -0.0072   
Franklin India ELSS Tax Saver Fund - Direct